In [5]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.chart import BarChart, Reference, PieChart, LineChart
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime

# Función para crear datos de ejemplo con la estructura sugerida - CORREGIDA
def cargar_datos():
    # Número total de mensajes
    total_mensajes = 1200  # 1000 para abril, 200 para mayo
    
    # Generar IDs para todos los mensajes
    ids = list(range(1, total_mensajes + 1))
    
    # Fechas para abril (4 semanas) y mayo (1 semana)
    fechas_abril = list(pd.date_range(start='2023-04-01', end='2023-04-30'))
    fechas_mayo = list(pd.date_range(start='2023-05-01', end='2023-05-07'))
    
    # Crear lista de fechas con la proporción correcta
    fechas_pool = fechas_abril * 5 + fechas_mayo * 10  # Crear un pool más grande del necesario
    np.random.shuffle(fechas_pool)
    fechas_envio = fechas_pool[:total_mensajes]
    
    # Crear campañas individuales y grupos de campaña
    campanas_individuales = []
    grupos_campana = []
    dias_semana = []
    horas_envio = []
    estados_envio = []
    clicks = []
    pedidos = []
    
    # Procesar cada mensaje individualmente para asegurar consistencia
    for fecha in fechas_envio:
        # Determinar grupo de campaña y campaña individual
        if fecha.month == 4:
            # En abril, varias campañas con la misma plantilla
            semana = (fecha.day - 1) // 7 + 1
            campanas_individuales.append(f"Difusión Abril Semana {semana}")
            grupos_campana.append("Plantilla Abril")
            
            # Rendimiento de la plantilla de abril
            estado = np.random.choice(['Entregado', 'Error', 'Pendiente'], p=[0.75, 0.20, 0.05])
            click = np.random.choice([True, False], p=[0.20, 0.80])
            pedido = False
            if click and estado == 'Entregado':
                pedido = np.random.choice([True, False], p=[0.40, 0.60])
        else:
            # En mayo, una única campaña
            campanas_individuales.append("Difusión Mayo")
            grupos_campana.append("Plantilla Mayo")
            
            # Rendimiento de la plantilla de mayo (mejor)
            estado = np.random.choice(['Entregado', 'Error', 'Pendiente'], p=[0.85, 0.12, 0.03])
            click = np.random.choice([True, False], p=[0.30, 0.70])
            pedido = False
            if click and estado == 'Entregado':
                pedido = np.random.choice([True, False], p=[0.60, 0.40])
        
        # Registrar estado y conversión
        estados_envio.append(estado)
        clicks.append(click)
        pedidos.append(pedido)
        
        # Agregar día de la semana y hora
        dias_semana.append(fecha.strftime('%A'))
        horas_envio.append(f"{np.random.randint(8, 20)}:00")
    
    # Verificar que todos los arrays tengan el mismo tamaño
    array_lengths = [
        len(ids), len(fechas_envio), len(campanas_individuales), len(grupos_campana),
        len(dias_semana), len(horas_envio), len(estados_envio), len(clicks), len(pedidos)
    ]
    
    assert len(set(array_lengths)) == 1, f"Arrays tienen tamaños diferentes: {array_lengths}"
    
    # Crear DataFrame
    datos = {
        'id_mensaje': ids,
        'fecha_envio': fechas_envio,
        'campana_individual': campanas_individuales,
        'grupo_campana': grupos_campana,
        'dia_semana': dias_semana,
        'hora_envio': horas_envio,
        'estado_envio': estados_envio,
        'click_boton': clicks,
        'pedido_completado': pedidos
    }
    
    return pd.DataFrame(datos)

# Función para crear resúmenes por grupo de campaña y por campaña individual
def crear_resumenes(df):
    # 1. Resumen por grupo de campaña (plantilla)
    resumen_grupo = crear_resumen_por_campo(df, 'grupo_campana')
    
    # 2. Resumen por campaña individual
    resumen_individual = crear_resumen_por_campo(df, 'campana_individual')
    
    # 3. Resumen por día de la semana
    resumen_dia = crear_resumen_por_campo(df, 'dia_semana')
    
    # 4. Resumen por hora de envío
    resumen_hora = crear_resumen_por_campo(df, 'hora_envio')
    
    return {
        'grupo': resumen_grupo,
        'individual': resumen_individual,
        'dia': resumen_dia,
        'hora': resumen_hora
    }

def crear_resumen_por_campo(df, campo_agrupacion):
    # Resumen básico por campo especificado
    agrupado = df.groupby(campo_agrupacion)
    
    # Estadísticas de envío
    estados = pd.crosstab(df[campo_agrupacion], df['estado_envio'])
    total_envios = estados.sum(axis=1)
    
    # Conversiones (solo sobre mensajes entregados)
    df_entregados = df[df['estado_envio'] == 'Entregado']
    entregados = df_entregados.groupby(campo_agrupacion).size()
    clicks = df_entregados.groupby(campo_agrupacion)['click_boton'].sum()
    pedidos = df_entregados.groupby(campo_agrupacion)['pedido_completado'].sum()
    
    # Crear DataFrame de resumen
    resumen = pd.DataFrame({
        'Total_Envios': total_envios,
        'Entregados': estados.get('Entregado', pd.Series(0, index=total_envios.index)),
        'Errores': estados.get('Error', pd.Series(0, index=total_envios.index)),
        'Pendientes': estados.get('Pendiente', pd.Series(0, index=total_envios.index)),
        'Tasa_Entrega (%)': (estados.get('Entregado', pd.Series(0, index=total_envios.index)) / total_envios * 100).round(2),
        'Clicks': clicks.fillna(0).astype(int),
        'Pedidos_Completados': pedidos.fillna(0).astype(int),
        'Tasa_Clicks (%)': (clicks.fillna(0) / entregados * 100).round(2),
        'Tasa_Conversion (%)': (pedidos.fillna(0) / clicks.fillna(1) * 100).round(2)
    })
    
    return resumen

# Función para crear el reporte Excel con múltiples hojas y análisis
def crear_excel_report(df, resumenes, nombre_archivo='reporte_campanas_completo.xlsx'):
    wb = Workbook()
    
    # 1. Hoja de datos detallados
    ws_datos = wb.active
    ws_datos.title = "Datos_Detallados"
    
    for r in dataframe_to_rows(df, index=False, header=True):
        ws_datos.append(r)
    
    aplicar_formato_tabla(ws_datos, len(df), len(df.columns))
    
    # 2. Hoja de resumen por grupo de campaña (plantilla)
    crear_hoja_resumen(wb, resumenes['grupo'], "Resumen_Plantillas", 
                     "Análisis por Plantilla de Campaña")
    
    # 3. Hoja de resumen por campaña individual
    crear_hoja_resumen(wb, resumenes['individual'], "Resumen_Campañas_Ind", 
                     "Análisis por Campaña Individual")
    
    # 4. Hoja de análisis por día de la semana
    crear_hoja_resumen(wb, resumenes['dia'], "Análisis_Día_Semana", 
                     "Rendimiento por Día de la Semana")
    
    # 5. Hoja de análisis por hora
    crear_hoja_resumen(wb, resumenes['hora'], "Análisis_Hora", 
                     "Rendimiento por Hora de Envío")
    
    # 6. Hoja de gráficos comparativos
    crear_hoja_graficos(wb, resumenes)
    
    # Guardar el libro de trabajo
    wb.save(nombre_archivo)
    print(f"Reporte guardado como {nombre_archivo}")

def aplicar_formato_tabla(ws, filas, columnas):
    # Dar formato al encabezado
    for col in range(1, columnas + 1):
        cell = ws.cell(row=1, column=col)
        cell.font = Font(bold=True)
        cell.fill = PatternFill("solid", fgColor="D9D9D9")
        cell.alignment = Alignment(horizontal='center')
    
    # Ajustar ancho de columnas
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            if cell.value:
                try:
                    max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
        adjusted_width = max_length + 2
        ws.column_dimensions[column].width = adjusted_width

def crear_hoja_resumen(wb, df_resumen, nombre_hoja, titulo):
    ws = wb.create_sheet(title=nombre_hoja)
    
    # Añadir título
    ws.append([titulo])
    ws.merge_cells(f'A1:J1')
    titulo_cell = ws.cell(row=1, column=1)
    titulo_cell.font = Font(bold=True, size=14)
    titulo_cell.alignment = Alignment(horizontal='center')
    
    # Añadir datos con un espacio después del título
    for r_idx, r in enumerate(dataframe_to_rows(df_resumen, index=True, header=True)):
        ws.append(r)
        
    # Dar formato
    aplicar_formato_tabla(ws, len(df_resumen) + 2, len(df_resumen.columns) + 1)
    
    return ws

def crear_hoja_graficos(wb, resumenes):
    ws = wb.create_sheet(title="Gráficos_Comparativos")
    
    # 1. Gráfico de barras: Comparación de tasas de entrega por plantilla
    bar_chart = BarChart()
    bar_chart.title = "Tasa de Entrega por Plantilla"
    bar_chart.y_axis.title = "Porcentaje"
    bar_chart.x_axis.title = "Plantilla"
    
    # Copiar datos temporales para el gráfico
    ws.append(["Plantilla", "Tasa Entrega", "Tasa Clicks", "Tasa Conversión"])
    
    row = 2
    for idx, datos in resumenes['grupo'].iterrows():
        ws.append([idx, datos['Tasa_Entrega (%)'], datos['Tasa_Clicks (%)'], datos['Tasa_Conversion (%)']])
        row += 1
    
    data = Reference(ws, min_col=2, max_col=4, min_row=1, max_row=row-1)
    cats = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=row-1)
    
    bar_chart.add_data(data, titles_from_data=True)
    bar_chart.set_categories(cats)
    bar_chart.height = 15
    bar_chart.width = 20
    
    ws.add_chart(bar_chart, "A10")
    
    # 2. Gráfico de línea: Rendimiento por día de la semana
    line_chart = LineChart()
    line_chart.title = "Rendimiento por Día de la Semana"
    line_chart.y_axis.title = "Porcentaje"
    line_chart.x_axis.title = "Día"
    
    # Crear datos temporales ordenados por día
    dias_orden = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    dias_esp = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
    
    ws.append([])  # Espacio
    ws.append(["Día", "Tasa Entrega", "Tasa Clicks", "Tasa Conversión"])
    
    row = row + 3
    start_row = row
    
    # Ordenar por día de la semana
    df_dias = resumenes['dia'].copy()
    if 'dia_semana' not in df_dias.columns:
        df_dias = df_dias.reset_index()
    
    # Mapeo para ordenar días
    df_dias['dia_orden'] = pd.Categorical(
        df_dias['dia_semana' if 'dia_semana' in df_dias.columns else 'index'],
        categories=dias_orden,
        ordered=True
    )
    df_dias = df_dias.sort_values('dia_orden')
    
    for idx, datos in df_dias.iterrows():
        dia = datos['dia_semana'] if 'dia_semana' in df_dias.columns else idx
        if dia in dias_orden:
            dia_esp = dias_esp[dias_orden.index(dia)]
            ws.append([dia_esp, 
                      datos['Tasa_Entrega (%)'], 
                      datos['Tasa_Clicks (%)'], 
                      datos['Tasa_Conversion (%)']])
            row += 1
    
    data = Reference(ws, min_col=2, max_col=4, min_row=start_row-1, max_row=row-1)
    cats = Reference(ws, min_col=1, max_col=1, min_row=start_row, max_row=row-1)
    
    line_chart.add_data(data, titles_from_data=True)
    line_chart.set_categories(cats)
    line_chart.height = 15
    line_chart.width = 20
    
    ws.add_chart(line_chart, "A30")
    
    return ws

# Ejecutar el análisis completo
df = cargar_datos()
resumenes = crear_resumenes(df)
crear_excel_report(df, resumenes)

AssertionError: Arrays tienen tamaños diferentes: [1200, 220, 220, 220, 220, 220, 220, 220, 220]

In [6]:
import openpyxl

# Crear un nuevo libro de trabajo
wb = openpyxl.Workbook()
hoja = wb.active

# Insertar algunos valores
hoja['A1'] = 10
hoja['A2'] = 20

# Insertar una fórmula de suma
hoja['A3'] = '=SUM(A1:A2)'

# Guardar el libro
wb.save('ejemplo_formula.xlsx')